In [ ]:
# Data Manipulation
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Natural Language Processing
import re
import string
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

# Machine Learning
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#Loading the data from a csv file
data = pd.read_csv("comments_1st.csv", encoding="ISO-8859-1")
comment_column = data.filter(['comments'])

# Initialize VADER sentiment analyzer
sid = SentimentIntensityAnalyzer()

# Calculate sentiment score for each comment
def get_sentiment_score(comment):
    return sid.polarity_scores(comment)['compound']

# Add a new column 'score' with sentiment scores
comment_column['score'] = comment_column['comments'].apply(get_sentiment_score)

In [ ]:
# Function to map score to sentiment label
def map_score_to_sentiment(score):
    if score > 0.5:
        return 'positive'
    elif score < 0.0:
        return 'negative'
    else:
        return 'neutral'

# Add a new column 'sentiment' with sentiment labels
comment_column['sentiment'] = comment_column['score'].apply(map_score_to_sentiment)
comment_column.head(5)


In [ ]:
# Plotting the data
sns.countplot(x='sentiment', data=comment_column)
plt.title

In [ ]:
# Conveting our target column "Sentiment" into numerical format. Positve to 2 , Neutral to 1 and negative to 0
comment_column.sentiment.replace("positive", 2, inplace=True)
comment_column.sentiment.replace("neutral", 1, inplace=True)
comment_column.sentiment.replace("negative", 0, inplace=True)
comment_column.head(5)

In [ ]:
# Text Preprocessing

def preprocess_text(text):
    """
    Preprocesses a given text by:
    1. Removing URLs
    2. Removing HTML tags
    3. Removing noise texts
    4. Converting text to lowercase
    5. Removing punctuation
    6. Removing numbers
    7. Tokenization
    8. Removing stopwords
    9. Stemming
    """
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove noise texts (you can customize this based on your data)
    noise_list = ['@username', 'RT', '&amp;', '...', 'etc.']
    for noise in noise_list:
        text = text.replace(noise, '')
    
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]
    
    # Join the stemmed tokens back into a single string
    preprocessed_text = ' '.join(stemmed_tokens)
    
    # Ensure the preprocessed text contains only alphabetic characters
    preprocessed_text = re.sub(r'[^a-zA-Z\s]', '', preprocessed_text)
    
    return preprocessed_text.strip()  # Remove leading and trailing whitespaces

# Preprocess the comments
comment_column['preprocessed_comments'] = comment_column['comments'].apply(preprocess_text)
comment_column.head(5)

In [ ]:
# Convert preprocessed comments to TF-IDF representation
# Initialize TfidfVectorizer to create TF-IDF representation
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the preprocessed comments to create TF-IDF representation
tfidf_representation = tfidf_vectorizer.fit_transform(comment_column['preprocessed_comments'])


In [ ]:
# Defining input and target variable
X = tfidf_vectorizer.fit_transform(comment_column['preprocessed_comments'])
y = comment_column['sentiment']

# Splitting the data into training and testing data set
test_size = 0.2
seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)


In [ ]:
# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_classifier = RandomForestClassifier(random_state=7)
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

In [ ]:
# Initialize the logistic regression classifier
classifier = RandomForestClassifier(**best_params)

# Train the classifier on the training data
classifier.fit(X_train, y_train)

# Make predictions on the test data
y_pred = classifier.predict(X_test)

# Print classification report
print(classification_report(y_test, y_pred))
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"F1-score = {f1}")

In [ ]:
# Testing the Classifying with some comments

# Preprocess a single comment
def preprocess_single_comment(comment):
    preprocessed_comment = preprocess_text(comment)
    return preprocessed_comment

# Testing comments
test_comments = input("Enter your comment here: ")

# Preprocess test comments
preprocessed_test_comments = [preprocess_single_comment(comment) for comment in test_comments]

# Vectorize test comments
tfidf_representation_test = tfidf_vectorizer.transform(preprocessed_test_comments)

# Make predictions on test comments
predictions = classifier.predict(tfidf_representation_test)

# Interpret predictions
for comment, prediction in zip(test_comments, predictions):
    sentiment_label = ['negative', 'neutral', 'positive'][prediction]
    print(f"Comment: {comment}: {sentiment_label} comment")